# ACh Markov model

Model and parameters taken from Lape, Colquhoun et al 2008 Nature
Markov model: Figure 5a
Parameters: Supplementary Table 2, ACh -100 mV fit



In [1]:
using Graphs

INFO: Recompiling stale cache file /Users/gergobohner/.julia/lib/v0.4/DataStructures.ji for module DataStructures.


In [90]:
# Initialize the graph
V = Array(ExVertex,0)
E = Array(ExEdge, 0)
g = Graphs.graph(V,[],is_directed=true)

# Fill up the vertices and edges
a = []
vertex_names = ["R","AR","A2R", "AF", "A2F", "ARs", "A2Rs"] 
for i1 = 1:length(vertex_names)
    a = make_vertex(g, vertex_names[i1])
    add_vertex!(g, a)
end

edge_names = ["kp1","km1","kp2","km2","δ1", "γ1", "δ2", "γ2", "β1", "α1", "β2", "α2", "kFp2", "kF-2"]
edge_vertices = [(1,2), (2,1), (2,3), (3,2), (2,4), (4,2), (3,5), (5,3), (4,6), (6,4), (5,7), (7,5), (4,5), (5,4)]
edge_values = [5.42*1e8, 9480, 2.71*1e8, 18960, 295000, 128000, 22900, 8400, 40, 17700, 87700, 2560, 2.03*1e8, 6480]
edge_values = convert(Array{Float64,1}, edge_values)
scales_with_ligand = convert(Array{Float64,1}, ([1 0 1 0 0 0 0 0 0 0 0 0 1 0])[:])
for i1 = 1:length(edge_names)
    a = make_edge(g, g.vertices[edge_vertices[i1][1]], g.vertices[edge_vertices[i1][2]])
    a.attributes["name"] = edge_names[i1]
    a.attributes["value"] = edge_values[i1]
    a.attributes["scales_with_ligand"] = scales_with_ligand[i1]
    add_edge!(g, a)
end

In [147]:
# Make the transition matrix
T = zeros(length(g.vertices), length(g.vertices))
# Make the ligand scaling matrix
dT = zeros(size(T))

for e in g.edges
    T[e.target.index, e.source.index] = e.attributes["value"]
    dT[e.target.index, e.source.index] = e.attributes["scales_with_ligand"]
end



In [156]:
# Define Popen for a ligand concentration (in Mol) and a transition matrix
function Popen(c::Float64, T::AbstractArray, dT::AbstractArray, openstates::Vector)
    Tlig = T.*(1-dT) + c.*T.*dT;
    # Make Tlig a valid transition matrix by making sure columns sum to 1
    for t1 = 1:size(Tlig,1) Tlig[t1,t1] = -sum(Tlig[:,t1]) end
    D, V = eig(Tlig) # non-ligand scaling + ligandscaling parts of the transition matrix
    assert(minimum(abs(D[:]))<1e-6) # There is a near 0 eigenvalue
    
    Vstat = V[:, sortperm(abs(D))[1]]
    
    return Vstat'*openstates*(1-2*(Vstat[1]<0)), D, Vstat
end
    


Popen (generic function with 1 method)

In [161]:
for c in logspace(-3, -1, 8)*1e-3
    op, D, Vstat = Popen(c, T, dT, [0, 0, 0, 0, 0, 1.0, 1.0])
end

In [162]:
Vstat

7-element Array{Float64,1}:
 0.000868651
 0.00496634 
 0.00906962 
 0.0111539  
 0.0291746  
 2.52065e-5 
 0.999458   

In [86]:
T.*(1-dT)

7x7 Array{Float64,2}:
    0.0       0.0     0.0       0.0      0.0   0.0      0.0
 9480.0       0.0     0.0  295000.0      0.0   0.0      0.0
    0.0   18960.0     0.0       0.0  22900.0   0.0      0.0
    0.0  128000.0     0.0       0.0      0.0  40.0      0.0
    0.0       0.0  8400.0    6480.0      0.0   0.0  87700.0
    0.0       0.0     0.0   17700.0      0.0   0.0      0.0
    0.0       0.0     0.0       0.0   2560.0   0.0      0.0

# Get MWC parameters (K and c) without knowing n

In [139]:
Popen(1e-19, T, dT, [0, 0, 0, 0, 0, 1.0, 1.0])

1-element Array{Float64,1}:
 2.35754e-14

In [175]:
op, D, Vstat = Popen(1e-23, T, dT, [0, 0, 0, 0, 0, 1.0, 1.0])

([2.9777601930061688e-21],[-5.820766091346741e-11,-429668.0212776196,-2818.859327714458,-17733.1193946658,-107640.82152541728,-261.88241804732166,-39097.296056535364],[-1.0,-5.717299332272742e-19,0.0,-1.3176589077299934e-18,0.0,-2.9777601930061688e-21,0.0])

In [174]:
# 1/(1+K) = op
log10K = log10(1-op) - log10(op)

1-element Array{Float64,1}:
 10.5261

In [177]:
# c is ratio of sensor affinity in open versus non-open state (here -> interpret as flipped vs non-flipped)
log10(0.13/43)

-2.5195251032727497